In [ ]:

#Import processing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import os, warnings, gc
import scipy.stats as stats
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

warnings.filterwarnings('ignore')

#Time series libraries
from scipy import stats
from pandas.plotting import lag_plot, autocorrelation_plot
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import pacf, plot_acf,plot_pacf
from pandas.plotting import autocorrelation_plot
import statsmodels.api as sm

from fbprophet import Prophet
from fbprophet.diagnostics import performance_metrics

In [ ]:
df = pd.read_csv('../input/electric-power-consumption-data-set/household_power_consumption.txt', sep = ';', header=0, low_memory=False, na_values = '?')
#df = pd.read_csv('../input/electric-power-consumption-data-set/household_power_consumption.txt', sep = ';', header=0, low_memory=False, infer_datetime_format=True, parse_dates={'datetime':[0,1]}, index_col=['datetime'],na_values = '?',dtype=float)

In [ ]:
df.head()

In [ ]:
#Dataframe datatypes
df.dtypes

In [ ]:
# Replacing the irregularities in the dataset with nan
df.drop(columns=['Time'],inplace=True)
df.replace('?', np.nan, inplace=True)

In [ ]:
# Checking for Null values
df.isnull().sum()

In [ ]:
# filling up the missing values with the mean values of the columns 
for columns in df.iloc[:, 1:]:
  df[columns].fillna(value = df[columns].mean(), inplace = True)
pass
df.isnull().sum()

In [ ]:
df.head(3)

In [ ]:
df['Date']= pd.to_datetime(df['Date'])
df.set_index("Date",inplace=True)

In [ ]:
# resample data to daily
daily_groups = df.resample('D')
daily_data = daily_groups.sum()

In [ ]:
df.isnull().sum()

In [ ]:
plt.figure(figsize=(14,7))
plt.plot(df.index,df.Global_active_power,'-')
plt.grid()
plt.xlabel('global active power')
plt.ylabel('Time')

In [ ]:
corr1 = np.corrcoef(df['Global_active_power'],df['Global_reactive_power'])
corr1
#stats.pearsonr(df['Global_active_power'],df['Global_reactive_power'])

In [ ]:
corr2 = np.corrcoef(df['Global_intensity'],df['Voltage'])
corr2

In [ ]:
df1 = df.copy()

In [ ]:
df1.head(3)

In [ ]:
df1 = df1.iloc[:,0:1]

In [ ]:
df1.head()

In [ ]:
df1.reset_index(inplace=True)

In [ ]:
df1.head()

In [ ]:
df1['Date']= pd.to_datetime(df1['Date'])

In [ ]:
df1 = df1.rename(columns={"Date": "ds", "Global_active_power": "y"})

In [ ]:
df1.head()

In [ ]:
train =  df1.iloc[:len(df1)-365,:]
test = df1.iloc[-365:,:]

In [ ]:
train.head()

In [ ]:
model = Prophet()

In [ ]:
model.fit(train)

In [ ]:
#future = model.make_future_dataframe(periods=365, freq='D')
forecast = model.predict(test)
forecast.head(3)

In [ ]:
model.plot_components(forecast)

In [ ]:
performance_metrics(model)

In [ ]:
df2 = df.copy()

In [ ]:
df2.head(3)

In [ ]:
df1 = df1.rename(columns={"datetime": "ds", "Global_active_power": "y", 
                          "Global_reactive_power": 'add1',
                         "Voltage":"add2",
                         "Global_intensity":"add3",
                         "Sub_metering_1":"add4",
                         "Sub_metering_2":"add5",
                         "Sub_metering_3":"add6"})

In [ ]:
new_train =  df1.iloc[:len(df1)-365,:]
new_test = df1.iloc[-365:,:]

In [ ]:
model_new = Prophet()

In [ ]:
model_new.add_regressor('add1')
model_new.add_regressor('add2')
model_new.add_regressor('add3')
model_new.add_regressor('add4')
model_new.add_regressor('add5')
model_new.add_regressor('add6')

In [ ]:
model_new.fit(new_train)

In [ ]:
#future_data = model.make_future_dataframe(periods=365, freq='D')
new_forecast = model_new.predict(new_test)
new_forecast.head(3)

In [ ]:
model_new.plot(new_forecast)

In [ ]:
model_new.plot_components(new_forecast)